In [ ]:
import sys

import numpy as np
import torch
import random

import matplotlib.pyplot as plt


import torch
from torch import nn, optim
from torch import load
from torch.nn import functional as F
from torch import autograd

from torchvision import datasets

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import time

import sys
from pathlib import Path

#from numba import njit

from tqdm import tqdm

import os

import gc

import torchntk.autograd as ezntk

import scipy.ndimage as ndimage

from cleverhans.torch.attacks.fast_gradient_method import fast_gradient_method
#from cleverhans.torch.attacks.projected_gradient_descent import (
#    projected_gradient_descent,
#)
from scipy.stats import kendalltau

from utils2 import process_Cifar10
from einops import rearrange

import torch
import torch.nn as nn
import os

from scipy.stats import kendalltau, spearmanr
from joblib import dump, load

In [ ]:
train, test, combined = process_Cifar10('./')
SEED = 0

In [ ]:

__all__ = [
    "ResNet",
    "resnet18",
    "resnet34",
    "resnet50",
]


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        padding=dilation,
        groups=groups,
        bias=False,
        dilation=dilation,
    )


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError("BasicBlock only supports groups=1 and base_width=64")
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(
        self,
        inplanes,
        planes,
        stride=1,
        downsample=None,
        groups=1,
        base_width=64,
        dilation=1,
        norm_layer=None,
    ):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.0)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(
        self,
        block,
        layers,
        num_classes=10,
        zero_init_residual=False,
        groups=1,
        width_per_group=64,
        replace_stride_with_dilation=None,
        norm_layer=None,
    ):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError(
                "replace_stride_with_dilation should be None "
                "or a 3-element tuple, got {}".format(replace_stride_with_dilation)
            )
        self.groups = groups
        self.base_width = width_per_group

        # CIFAR10: kernel_size 7 -> 3, stride 2 -> 1, padding 3->1
        self.conv1 = nn.Conv2d(
            3, self.inplanes, kernel_size=3, stride=1, padding=1, bias=False
        )
        # END

        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(
            block, 128, layers[1], stride=2, dilate=replace_stride_with_dilation[0]
        )
        self.layer3 = self._make_layer(
            block, 256, layers[2], stride=2, dilate=replace_stride_with_dilation[1]
        )
        self.layer4 = self._make_layer(
            block, 512, layers[3], stride=2, dilate=replace_stride_with_dilation[2]
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(
            block(
                self.inplanes,
                planes,
                stride,
                downsample,
                self.groups,
                self.base_width,
                previous_dilation,
                norm_layer,
            )
        )
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(
                block(
                    self.inplanes,
                    planes,
                    groups=self.groups,
                    base_width=self.base_width,
                    dilation=self.dilation,
                    norm_layer=norm_layer,
                )
            )

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        z = x.reshape(x.size(0), -1)
        x = self.fc(z)

        return x
    
    def forward_cam_conv1(self, x):
        y = self.conv1(x)
        x = self.bn1(y)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        z = x.reshape(x.size(0), -1)
        x = self.fc(z)

        return x, y
    
    def forward_conv1(self, x):
        x = self.conv1(x)
        return x
    def forward_bn1(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        return x
    def forward_layer1(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        return x
    def forward_layer2(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        return x
    def forward_layer3(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x
    def forward_layer4(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        return x
    def forward_flat(self,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.reshape(x.size(0), -1)
        return x
        


def _resnet(arch, block, layers, pretrained, progress, device, **kwargs):
    model = ResNet(block, layers, **kwargs)
    if pretrained:
        script_dir = os.path.dirname(__file__)
        state_dict = torch.load(
            script_dir + "/state_dicts/" + arch + ".pt", map_location=device
        )
        model.load_state_dict(state_dict)
    return model


def resnet18(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet18", BasicBlock, [2, 2, 2, 2], pretrained, progress, device, **kwargs
    )

def resnet34(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-34 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet34", BasicBlock, [3, 4, 6, 3], pretrained, progress, device, **kwargs
    )


def resnet50(pretrained=False, progress=True, device="cpu", **kwargs):
    """Constructs a ResNet-50 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _resnet(
        "resnet50", Bottleneck, [3, 4, 6, 3], pretrained, progress, device, **kwargs
    )

In [ ]:
class ConvBNReLU(nn.Sequential):
    def __init__(self, in_planes, out_planes, kernel_size=3, stride=1, groups=1):
        padding = (kernel_size - 1) // 2
        super(ConvBNReLU, self).__init__(
            nn.Conv2d(
                in_planes,
                out_planes,
                kernel_size,
                stride,
                padding,
                groups=groups,
                bias=False,
            ),
            nn.BatchNorm2d(out_planes),
            nn.ReLU6(inplace=True),
        )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        layers = []
        if expand_ratio != 1:
            # pw
            layers.append(ConvBNReLU(inp, hidden_dim, kernel_size=1))
        layers.extend(
            [
                # dw
                ConvBNReLU(hidden_dim, hidden_dim, stride=stride, groups=hidden_dim),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            ]
        )
        self.conv = nn.Sequential(*layers)

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=10, width_mult=1.0):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        last_channel = 1280

        # CIFAR10
        inverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 1],  # Stride 2 -> 1 for CIFAR-10
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]
        # END

        # building first layer
        input_channel = int(input_channel * width_mult)
        self.last_channel = int(last_channel * max(1.0, width_mult))

        # CIFAR10: stride 2 -> 1
        features = [ConvBNReLU(3, input_channel, stride=1)]
        # END

        # building inverted residual blocks
        for t, c, n, s in inverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                stride = s if i == 0 else 1
                features.append(
                    block(input_channel, output_channel, stride, expand_ratio=t)
                )
                input_channel = output_channel
        # building last several layers
        features.append(ConvBNReLU(input_channel, self.last_channel, kernel_size=1))
        # make it nn.Sequential
        self.features = nn.Sequential(*features)

        # building classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.last_channel, num_classes),
        )

        # weight initialization
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out")
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.zeros_(m.bias)

    def forward(self, x):
        x = self.features(x)
        z = x.mean([2, 3])
        x = self.classifier(z)
        return x


def mobilenet_v2(pretrained=False, progress=True, device="cpu", **kwargs):
    """
    Constructs a MobileNetV2 architecture from
    `"MobileNetV2: Inverted Residuals and Linear Bottlenecks" <https://arxiv.org/abs/1801.04381>`_.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = MobileNetV2(**kwargs)
    if pretrained:
        script_dir = os.path.dirname(__file__)
        state_dict = torch.load(
            script_dir + "/state_dicts/mobilenet_v2.pt", map_location=device
        )
        model.load_state_dict(state_dict)
    return model

In [ ]:
class VGG(nn.Module):
    def __init__(self, features, num_classes=10, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        # CIFAR 10 (7, 7) to (1, 1)
        # self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.classifier = nn.Sequential(
            nn.Linear(512 * 1 * 1, 4096),
            # nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
        )
        self.fc = nn.Linear(4096, num_classes),
        
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        z = x.view(x.size(0), -1)
        x = self.classifier(z)
        return x, z

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == "M":
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    "A": [64, "M", 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "B": [64, 64, "M", 128, 128, "M", 256, 256, "M", 512, 512, "M", 512, 512, "M"],
    "D": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        "M",
    ],
    "E": [
        64,
        64,
        "M",
        128,
        128,
        "M",
        256,
        256,
        256,
        256,
        "M",
        512,
        512,
        512,
        512,
        "M",
        512,
        512,
        512,
        512,
        "M",
    ],
}


def _vgg(arch, cfg, batch_norm, pretrained, progress, device, **kwargs):
    if pretrained:
        kwargs["init_weights"] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        script_dir = os.path.dirname(__file__)
        state_dict = torch.load(
            script_dir + "/state_dicts/" + arch + ".pt", map_location=device
        )
        model.load_state_dict(state_dict)
    return model


def vgg11_bn(pretrained=False, progress=True, device="cpu", **kwargs):
    """VGG 11-layer model (configuration "A") with batch normalization
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    return _vgg("vgg11_bn", "A", True, pretrained, progress, device, **kwargs)

In [ ]:
MODELNAME = 'ResNet18'
modelname = 'resnet18'

#modelname = 'mobilenet_v2'
#modelname


In [ ]:
if MODELNAME == 'ResNet18':
    model = resnet18(device='cuda')
    model.load_state_dict(torch.load('./resnet18.pt'))
elif MODELNAME == 'ResNet34':
    model = resnet34(device='cuda')
    model.load_state_dict(torch.load('./resnet34.pt'))
elif MODELNAME == 'MobileNetV2':
    model = mobilenet_v2(device='cuda')
    model.load_state_dict(torch.load('./mobilenet_v2.pt'))
else:
    raise ValueError('not a selected model')

model.cuda()

In [ ]:
train_data = datasets.CIFAR10(
    root = '../DATA/',
    train = True,                          
    download = False,            
)


test_data = datasets.CIFAR10(
    root = '../DATA/', 
    train = False, 
    download=False,
)

In [ ]:
train_x = torch.tensor(train_data.data)
test_x = torch.tensor(test_data.data)

train_y = torch.tensor(train_data.targets)
test_y = torch.tensor(test_data.targets)

train_x_plot = torch.tensor(train_data.data).cpu().numpy()
test_x_plot = torch.tensor(test_data.data).cpu().numpy()

train_y_plot = torch.tensor(train_data.targets).cpu().numpy()
test_y_plot = torch.tensor(test_data.targets).cpu().numpy()

train_x = train_x/255
test_x = test_x/255

mean = torch.tensor([0.4914, 0.4822, 0.4465])
std = torch.tensor([0.2471, 0.2435, 0.2616])

test_x -= mean[None,None,None,:]
test_x /= std[None,None,None,:]

train_x -= mean[None,None,None,:]
train_x /= std[None,None,None,:]

train_y = train_y.float()
test_y = test_y.float()

train_x = train_x.to('cuda')
train_y = train_y.to('cuda')

test_x = test_x.to('cuda')
test_y = test_y.to('cuda')

train_x = rearrange(train_x,'b w h f -> b f w h')
test_x = rearrange(test_x,'b w h f -> b f w h')

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset

train_loader = DataLoader(train,batch_size=64,shuffle=False)
test_loader = DataLoader(test,batch_size=64,shuffle=False)
test_loader2 = DataLoader(test,batch_size=150,shuffle=False)
combined_loader = DataLoader(combined,batch_size=150,shuffle=False)

In [ ]:
model = model.eval()
model.to('cuda')
correct=torch.tensor([0,],device='cuda')
train_predictions = []
with torch.no_grad():
    for i,data in enumerate(train_loader):
        x, y = data
        x = x.cuda()
        y = y.cuda()
        #out, z = model(x)
        out = model(x)
        classify = torch.argmax(torch.softmax(out,dim=1),dim=1)
        num = torch.sum(classify==y)
        correct+=num
        train_predictions.append(classify.cpu().numpy())     
print('NN Train acc: {:.2f}'.format((100*correct.cpu().numpy()/len(train_y))[0]))

train_predictions=np.concatenate(train_predictions)

In [ ]:
model = model.eval()
correct=torch.tensor([0,],device='cuda')
predictions_model = []
confidences_model_right_class = []
confidences_model_guessed_class = []
confidences_all = []
NN_out = []
NN_logits = []
with torch.no_grad():
    for i,data in enumerate(test_loader):
        x, y = data
        x = x.cuda()
        y = y.cuda()
        out = model(x)
        NN_logits.append(out.cpu().numpy())
        classify = torch.argmax(torch.softmax(out,dim=1),dim=1)
        NN_out.append(torch.softmax(out,dim=1))
        for i in range(len(y)):
            confidences_model_right_class.append(torch.softmax(out,dim=1)[i][torch.tensor(y[i],dtype=torch.long)])
            confidences_model_guessed_class.append(torch.softmax(out,dim=1)[i][classify[i]])
        confidences_all.append(out)
        predictions_model.append(classify)
        num = torch.sum(classify==y)
        correct+=num
print('NN Test acc: {:.2f}'.format((100*correct.cpu().numpy()/len(test_y))[0]))
predictions_model = torch.cat(predictions_model)
confidences_model_right_class = torch.stack(confidences_model_right_class).cpu().numpy()
confidences_model_guessed_class = torch.stack(confidences_model_guessed_class).cpu().numpy()
confidences_all = torch.cat(confidences_all).cpu().numpy()
NN_out = torch.cat(NN_out).cpu().numpy()
test_predictions = np.argmax(NN_out,axis=1)

In [ ]:
NN_logits = np.concatenate(NN_logits)

In [ ]:
from scipy.optimize import minimize
from scipy.special import erf
from scipy.stats import norm
from math import log
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import r2_score

from joblib import dump, load

In [ ]:
train_labels_regression = train_y.cpu().numpy()
validation_labels = test_y.cpu().numpy()

# load NTK

In [ ]:
index_test=10_000
if modelname == 'resnet18':
    NTK = torch.load(f'../KERNELS/CV/fullNTK.pt',map_location='cpu')
    K0 = NTK[index_test::,index_test::].cpu().numpy()
    K1 = NTK[0:index_test,index_test::].cpu().numpy()
    K2 = NTK[0:index_test,0:index_test].cpu().numpy()
else:
    raise NotImplementedError
    #not included in this code release
    
    


K1 = K1 / (np.sqrt(np.diagonal(K0))[None,:]) / (np.sqrt(np.diagonal(K2))[:,None])
K0 = K0 / np.sqrt(np.diagonal(K0))[:,None] / np.sqrt(np.diagonal(K0))[None,:]

In [ ]:
LogKernelRegression_pNTK = SGDClassifier(loss='log_loss',penalty='l2',alpha=1e-4,fit_intercept=True,class_weight="balanced")

In [ ]:
def fit_and_scale(LogKernelRegression,K0,K1,validation_labels):
    LogKernelRegression.fit(K0,train_labels_regression)
    weights = LogKernelRegression.coef_
    intercept = LogKernelRegression.intercept_
    y = LogKernelRegression.predict_proba(K1)
    print((LogKernelRegression.predict(K1)==validation_labels.cpu().numpy()[0:len(y)]).sum()/len(y))
    return weights, intercept, y, LogKernelRegression

In [ ]:
def tau_comparison_final(X,Y):    
    
    correct_logit_mask = test_y.cpu().numpy().astype(int)
    
    
    X = np.array([softmax_numpy(X[i])[correct_logit_mask[i]] for i in range(len(X))])
    Y = np.array([softmax_numpy(Y[i])[correct_logit_mask[i]] for i in range(len(Y))])    

    
    mask = np.logical_or(np.logical_or(X==1,Y==1),np.logical_or(X==0,Y==0))
    X = X[~mask]
    Y = Y[~mask]
    
    plt.plot(X,Y,'.')
    
    Tau = kendalltau(X,Y).correlation
    return Tau

In [ ]:
#K1_weights, K1_intercept, y_logisticregression, LogKernelRegression_pNTK = fit_and_scale(LogKernelRegression_pNTK,K0,K1,test_y)


if modelname == 'resnet18':
    LogKernelRegression_pNTK = load('../kGLMs/CV/pNTK.joblib')
else:
    raise NotImplementedError
    #not included in the code release
    #LogKernelRegression_pNTK = load(f'XXX/large_model_NTKs/{modelname}/NTK_GLM.joblib')

K1_weights = LogKernelRegression_pNTK.coef_
K1_intercept = LogKernelRegression_pNTK.intercept_

K1_scaled = (K1 @ K1_weights.T) + K1_intercept

# test acc: ResNet18: 0.929
# test acc: ResNet34: 0.9283
# test acc: MobileNetV2: 0.9369

In [ ]:
def softmax_numpy(x):
    return np.exp(x) / np.sum(np.exp(x))

tau_comparison_final(K1_scaled,confidences_all)

#ResNet18 Tau: 0.7755961255438976
#ResNet34 Tau: 0.7858821360816239
#MobileNetV2 Tau: 0.7003158758871003


# Using fit, now visualize

In [ ]:
from scipy.optimize import minimize
from scipy.special import erf
from scipy.stats import norm
from math import log
from sklearn.metrics import r2_score

def numpy_softmax(x):
    return np.exp(x) / np.sum(np.exp(x),axis=1)[:,None]

def sigmoid(p):
    return 1/(1+np.exp(-p))

def logit(p,eps=0.0):
    return np.log((p+eps)/(1-p+eps))

def softmax_numpy(x):
    return np.exp(x) / np.sum(np.exp(x))

In [ ]:
def new_func(X,Theta):
    
    a=Theta[0]
    b=Theta[1]
    c=Theta[2]
    s=Theta[3]
    
    Y = np.exp((X-a)/b)
    Y = s*(Y/(1+Y)) + c
    return Y

def guesses(X,Y):
    c = np.quantile(Y,0.05)
    s = np.quantile(Y,0.90)-c
    DB = (0.5*s + c)
    a = 1*X[np.argmin(abs(Y-DB))]
    
    m_index = np.argsort(abs(Y-DB))[0:2]
    m = (Y[m_index[1]] - Y[m_index[0]]) / (X[m_index[1]] - X[m_index[0]])
    if m < 0:
        m=m*-1
    m=0.5

    b = s/(4*m)
    Theta = np.array([a,b,c,s])
    #print('initial Guess: ',Theta)
    return Theta

def new_loss(Theta,X,Y):
    delta=1.0
    Yhat = new_func(X,Theta)
    E=abs(Yhat-Y)
    maskh = E>delta
    HuberLoss=0
    HuberLoss+= np.sum((delta*E[maskh] - 0.5*delta))
    HuberLoss+= np.sum((delta*(Yhat-Y)[~maskh]**2))
    return HuberLoss


In [ ]:
from seaborn import kdeplot
def forward_fit_final(X,Y,method='Nelder-Mead',PLOT=True):
    #X = kGLM activations
    #Y = NN softmax vector
    mask_trueclass = []
    all_Xs = []
    all_Ys = []
    for class_ in range(10):
        downselect_mask =  test_y.cpu().numpy().astype(int) == class_
    
        X1 = X[downselect_mask]
        Y1 = Y[downselect_mask]
    
        for i in range(10):
            
            if i == class_:
                if class_!=0:
                    where = np.array([class_,class_-1],dtype=int)
                else:
                    where = np.array([class_,1],dtype=int)
            if i != class_:
                where = np.array([class_,i],dtype=int)

            X2=X1[:,where]
            how_many=len(X2)
            X2 = X2.flatten()
            Y2=Y1[:,where].flatten()
        
            x0 =  guesses(X2,Y2)
            res = minimize(new_loss,x0=x0,args=(X2,Y2),method=method)
            Theta = res.x
            DS = 0.5*Theta[3] +Theta[2]
            Yhat = new_func(X2,Theta)

            if i != class_:
                all_Xs.append(Yhat[1::2])
                all_Ys.append(Y2[1::2])
                mask_trueclass.append(np.zeros(len(Y2[1::2]),dtype=bool))
            if i == class_:
                all_Xs.append(Yhat[0::2])
                all_Ys.append(Y2[0::2])
                mask_trueclass.append(np.ones(len(Y2[0::2]),dtype=bool))
        
    mask_trueclass = np.concatenate(mask_trueclass)
#     for i in range(len(mask_trueclass)):
#         if mask_trueclass[i]:
#             continue
#         else:
#             if np.random.random() > 0.9:
#                 mask_trueclass[i] = True
    all_Xs = logit(np.array(all_Xs).flatten())
    all_Ys = logit(np.array(all_Ys).flatten())
    print(all_Xs.shape)
    print(all_Ys.shape)
    
    maskout = np.logical_or(np.logical_or(np.isnan(all_Xs),np.isinf(all_Xs)),np.logical_or(np.isnan(all_Ys),np.isinf(all_Ys)))
    all_Xs = all_Xs[~maskout]
    all_Ys = all_Ys[~maskout]
    mask_trueclass = mask_trueclass[~maskout]
    R_squared = r2_score(all_Xs,all_Ys,)
    center_mask1 = np.logical_and(all_Xs[mask_trueclass]>-5,all_Xs[mask_trueclass]<-0)
    center_mask2 = np.logical_and(all_Xs[~mask_trueclass]>-5,all_Xs[~mask_trueclass]<-0)
    if PLOT:
        #After Mapping
#         plt.plot(all_Xs[~mask_trueclass],all_Ys[~mask_trueclass],'.',alpha=0.05,)
#         plt.plot(all_Xs[mask_trueclass],all_Ys[mask_trueclass],'s',alpha=0.05,)
#         plt.plot([np.min(all_Ys),np.max(all_Ys)],[np.min(all_Ys),np.max(all_Ys)],'k--',alpha=1.0,label='parity')
#         plt.ylabel(f'All logits NN ',fontsize=14)
#         plt.xlabel(f'All logits kGLM',fontsize=14)
#         plt.title(f'ResNet18 Linearization',fontsize=14)
#         plt.ylim(np.min(all_Ys),np.max(all_Ys)+1)
#         plt.xlim(np.min(all_Xs),np.max(all_Xs)+1)
#         plt.plot(-1000,-1000,'.',alpha=1.0,color='tab:blue',label='Incorrect Class')
#         plt.plot(-1000,-1000,'.',alpha=1.0,color='tab:orange',label='Correct Class')
#         plt.legend()
#         plt.show()
        #plt.plot(all_Xs[~mask_trueclass][center_mask2],all_Ys[~mask_trueclass][center_mask2],'.',alpha=0.05,color='tab:blue')
        #plt.plot(all_Xs[mask_trueclass][center_mask1],all_Ys[mask_trueclass][center_mask1],'s',alpha=0.05,color='tab:orange')
        kdeplot(all_Xs[~mask_trueclass][::1000],all_Ys[~mask_trueclass][::1000],fill=True,color='tab:blue')
        kdeplot(all_Xs[mask_trueclass][::100],all_Ys[mask_trueclass][::100],fill=True,color='tab:orange')
        
        plt.plot([np.min(all_Ys),np.max(all_Ys)],[np.min(all_Ys),np.max(all_Ys)],'r--',alpha=1.0,label='parity')
        plt.ylabel(f'All logits NN ',fontsize=14)
        plt.xlabel(f'All logits kGLM',fontsize=14)
        plt.title(f'{MODELNAME} Linearization',fontsize=14)
        plt.ylim(-12,8)
        plt.xlim(-12,8)
        plt.plot(-1000,-1000,color='tab:blue',label='Incorrect Class Logit')
        plt.plot(-1000,-1000,color='tab:orange',label='Correct Class Logit')
        plt.text(-5,2,'R^2 = {:.3f}'.format(R_squared),fontsize=12)
        plt.legend()
       
        plt.show()
        
    
    
    return R_squared#, all_tau

In [ ]:
forward_fit_final(K1_scaled,numpy_softmax(confidences_all),)


#R2 ResNet18: 0.9139234434009396
#R2 ResNet34: 0.9202184987953692
#R2 MobileNetV2: 0.9119214895054525